### Raspagem de dados do perfil do LinkedIn

In [27]:
import warnings
warnings.filterwarnings("ignore")

In [28]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [29]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By

In [30]:
# Abrir o Chrome
driver = webdriver.Chrome()

In [31]:
# Com o Chrome aberto, acessar o link de login do linkedin
driver.get("https://www.linkedin.com/login")

In [32]:
# Na página de login preencher o campo email
email = driver.find_element(By.ID, 'username')
email.send_keys(os.getenv('EMAIL'))

In [33]:
# Na página de login preencher o campo senha
password = driver.find_element(By.ID, 'password')
password.send_keys(os.getenv('PASSWORD'))

In [34]:
# Enviar as informações de login
password.submit()

In [35]:
# Abrir o perfil

url = os.getenv('LINKEDIN_URL')
driver.get(url)


In [36]:
# captura todo o HTML atual da página aberta no navegador (a fim de encontrar as sessões do perfil)
page_source = driver.page_source

In [37]:
# Através da biblioteca BeautifulSoup, transforma o HTML em um objeto navegável
# Objetivo: Buscar, extrair e manipular dados de HTML facilmente
soup = BeautifulSoup(page_source, 'html.parser')

In [38]:
# Encontra a classe principal que mostra tudo do perfil
profile = soup.find('main', {'class': 'HuakhXxGQdSsPopKGKtFOGWUEchdcbsjw'})

In [39]:
# Trazer todas as sessões do perfil
sections = profile.find_all('section', {'class': 'artdeco-card'})

In [40]:
# Iterar as sessões para retornar uma lista de cada sessão individualmente
sections_text = [section.get_text() for section in sections]

In [41]:
# Limpar o html retornado para deixar somente dados importantes e não repetidos

import re

def clean_text(text):
    text = re.sub(r'\n+', '\n', text) #substitui oq repete por uma única linha
    text = re.sub(r'\t+', '\t', text)
    text = re.sub(r'\t\s+', ' ', text)
    text = re.sub(r'\n\s+', '\n', text)
    return text

In [42]:
# Aplica a limpeza nas nossas sessões
sections_text = [clean_text(section) for section in sections_text]

In [43]:
#Removendo palavras duplicadas

def remove_duplicates(text):
    # Divide o texto em linhas, separando pelo caractere de nova linha '\n'
    lines = text.split('\n')
    
    # Inicializa uma lista para armazenar as linhas processadas
    new_lines = []
    
    # Percorre cada linha do texto original
    for line in lines:
        # Verifica se a primeira metade da linha é igual à segunda metade
        if line[:len(line)//2] == line[len(line)//2:]:
            # Se for igual, adiciona apenas a primeira metade na lista (corrigido aqui)
            new_lines.append(line[:len(line)//2])
        else:
            # Caso contrário, adiciona a linha inteira
            new_lines.append(line)
    
    # Junta todas as linhas da lista em uma única string, separando por '\n'
    return '\n'.join(new_lines)

In [44]:
# Aplica a limpeza de duplicação nas nossas sessões
sections_text = [remove_duplicates(section) for section in sections_text]

### Análise de dados de perfil usando LLM

In [45]:
# Importa classes necessárias da biblioteca langchain para construir prompts e interagir com o modelo de linguagem Ollama
from langchain_ollama import ChatOllama
from langchain_core.prompts import (SystemMessagePromptTemplate,
                                    HumanMessagePromptTemplate,
                                    ChatPromptTemplate)
from langchain_core.output_parsers import StrOutputParser

# Define a URL base onde o serviço Ollama está rodando localmente
base_url = 'http://localhost:11434'

# Escolhe qual modelo de linguagem será usado; a linha comentada usa 'llama3.2:3b' e a ativa usa 'qwen2.5:7b'

model = 'llama3.2:3b'
#model = 'qwen2.5:7b'

# Inicializa o cliente do modelo Ollama com a URL base e o modelo escolhido
llm = ChatOllama(base_url=base_url, model=model)

# Define um template de mensagem do sistema, que funciona como uma instrução inicial para o modelo
# Aqui, o sistema informa que é um assistente de IA que responde perguntas sobre análise de perfil do LinkedIn,
# usando dados de texto fornecidos do perfil.
system = SystemMessagePromptTemplate.from_template("""
Você é um assistente de IA útil que responde a perguntas de usuários relacionadas à análise de perfil do LinkedIn com base nos dados de texto do perfil fornecidos.
""")

# Define uma função para fazer perguntas ao modelo de linguagem
def ask_llm(prompt):
    # Cria um template para a mensagem do usuário, contendo o prompt passado para a função
    prompt = HumanMessagePromptTemplate.from_template(prompt)
    
    # Combina a mensagem do sistema e do usuário numa lista de mensagens para o chat
    messages = [system, prompt]

    # Cria o template do chat com as mensagens do sistema e usuário
    template = ChatPromptTemplate(messages)

    # Constrói a cadeia de execução: envia o template para o modelo (llm) e depois usa um parser para obter a resposta como string simples
    qna_chain = template | llm | StrOutputParser()

    # Executa a cadeia (invoke) sem parâmetros adicionais e retorna a resposta do modelo
    return qna_chain.invoke({})


In [46]:
ask_llm("hello")

'Olá! Como posso ajudar você hoje? Você está procurando por alguma coisa específica sobre o seu perfil no LinkedIn ou precisa de ajuda para entender melhor seus perfis? Estou aqui para ajudar!'

In [47]:
# Define um template de prompt para enviar ao modelo, com instruções claras sobre o que extrair dos dados do perfil LinkedIn
template = """ 

Extraia e retorne as informações solicitadas dos dados do perfil do LinkedIn em um formato conciso, ponto a ponto (até 5 pontos). Evite preâmbulos ou qualquer contexto adicional.

### Dados do perfil do LinkedIn:
{}

### Informações a serem extraídas:

Extraia '{}' em marcadores, limitando a saída até 9 pontos. Forneça apenas os detalhes necessários.

Lembre-se, são dados de perfil do LinkedIn.

### Dados extraídos: """

# 'context' recebe o texto da primeira seção dos dados do perfil LinkedIn (assumindo que sections_text seja uma lista de textos)
context = sections_text[0]

# 'k' é a informação que queremos extrair do perfil — aqui, "Nome e Título"
k = "Nome e Título"

# Formata o template, inserindo o texto do perfil e a informação específica que queremos extrair
prompt = template.format(context, k)

# Chama a função 'ask_llm' que envia o prompt ao modelo de linguagem e recebe a resposta
response = ask_llm(prompt)


In [48]:
print(response)

*   Nome: Sarah Batagioti
*   Título: Desenvolvedora Full Stack 
*   Habilidades:
    *   Python
    *   JavaScript
    *   HTML
    *   CSS
    *   React
    *   MySQL
*   Localização: São José dos Campos, São Paulo, Brasil


In [49]:
# Extrai os títulos das seções dos textos do perfil LinkedIn para usar como chaves identificadoras# Percorre cada título de cada sessão

section_keys = ['Nome e título']
for section in sections_text[1:]:
    section_keys.append(section.strip().split('\n')[0])

section_keys

['Nome e título', '', '', '', '', '']

In [50]:
# Gera respostas para cada seção do perfil LinkedIn usando prompts personalizados e armazena em um dicionário
responses = {}

for k, context in zip(section_keys, sections_text):
    prompt = template.format(context, k)   # Prepara o prompt com o texto da seção e o título correspondente
    response = ask_llm(prompt=prompt)      # Envia o prompt ao modelo e obtém a resposta
    responses[k] = response                 # Salva a resposta no dicionário com a chave do título da seção

In [ ]:
# Criar arquivo json com as informações do perfil 

import json

with open('linkedin_profile_data.json', 'w', encoding='utf-8') as f:
    json.dump(responses, f, indent=4, ensure_ascii=False)